# MDA_HW1_106061218 李丞恩

## 一. 安裝pyspark

本次的作業要實作利用MapReduce進行矩陣乘法。在安裝pyspark時我遇到了一些麻煩。我先按照提供的安裝方法的pdf檔實作，卻發現無法使用使用pip下載時會出現各種相容性的問題，後來google以後，發現可能是因為我電腦的python版本是3.7，以及一些pip，pip3版本問題的緣故。但我其他專題可能會使用更高版本的python，因此我用Anaconda創建一個python3.6虛擬環境後，使用

conda install pyspark

後，再使用

conda install jupyter

安裝jupyter notebook，並用指令

spark

開啟jupyter notebook，方能成功安裝並使用pyspark。

## 二. 實作方法

首先引進需要的兩個東西

In [ ]:
from pyspark import SparkConf, SparkContext

建議每次開啟都先執行下面這一行，這是為了避免記憶體裡還有sparkcontext仍在執行。

In [14]:
sc.stop()

首先定義mapper函數。根據500input.txt的結構，Map的方法是：我們先用分號作為每列之間的分隔，再用逗號作為每一列之中文字的分隔。假設$M$是一個$i\times j$的矩陣，$N$是一個$j\times k$的矩陣，則我們都要以$j$為key，剩下三者作為value。我將每一行除了$j$以外的剩下三者--$M$(或$N$)，$i$(或$k$)，以及$M_{ij}$(或$N_{jk}$)組成一個tuple。每一列都這麼做以後，再加到之後會轉為的RDD的list內。最後會生成形如
$$\bigg[\big(0, (M, 0, 51)\big),...\bigg]-(1)$$的結構。

In [15]:
def mapper1(line):
    wordlist=line.split(" ")
    maplist=[]
    for item in wordlist:
        s = item.split(",")
        if s[0]=="M":
            key=int(s[2])  # select column as key
            value=(s[0],int(s[1]),int(s[3])) # construct tuple
        elif s[0]=="N":
            key=int(s[1]) # select column as key
            value=(s[0],int(s[2]),int(s[3])) # construct tuple
        maplist.append((key,value)) # add to map
    return maplist

In [16]:
def reducer1(x,y):
    return x+y

以下的block，前兩行詢問班上同學後，為呼叫sparkconf與sparkcontext的方式，不必太過糾結意思。第三行是將500input讀進一個檔案內rdd檔案內，而collect()是將rdd轉為list顯示出來。由於每次都會花很久的時間，寫信問助教後，說可以使用第四行的寫法讓它只顯示前十個。由於我這裡只是把collect()當成debug用的方法，因此確認沒問題後就註解掉了。

In [17]:
conf = SparkConf().setMaster("local").setAppName("wordcount") # call sparkconf
sc = SparkContext(conf=conf) # call sparkcontext
lines = sc.textFile("500input.txt").flatMap(mapper1) # read txt file
# print(lines.collect()[:10])

我的計畫是使用兩次map加一次reduce。第一次map如上所述，接著，原本我寫一個reduce的函數以形成如
$$\bigg[\bigg(0,\big((M,0,51), (N,0,73)\big)\bigg),......\bigg]-(2)$$
的結構，不過卻因為不知道如何操作rdd資料結構內作為變數的x與y。幸好經由班上熱心的同學指點，他說可以將M與N拆開後，再使用join來達成我想做的事情。接著，將這個結果再使用第二次的map，並希望能生成形如下列的結構：
$$\bigg[\big((0,0),51\times73\big),......\bigg]-(3)$$
也就是說，(2)式中的value有兩個tuple，要相乘在一起的數值在$M$的行數，與在$N$的列數組合成一個新的key，也就是把在$P=M\times N$中，相加後會等於$P_{ik}$的座標當成key。而value就是兩個值相乘。最後，使用我寫的reduce function，就可以將key相同的value們加在一起，從而得到正確的$P_{ik}$。也就是
$$\bigg[\big((i,k),P_{ik}\big),......\bigg]-(4)$$

因此我們先用filter的方法，將python中隱函數的方法作為參數，將$M$與$N$拆開來。換句話說，就是參考(1)中value的第一個值作為拆開的依據。

In [18]:
M = lines.filter(lambda x : "M" in x[1][0])
N = lines.filter(lambda x : "N" in x[1][0])

以下是debug的遺跡，不必多加理會。說明$M$與$N$有有確實拆開。

In [9]:
# print(M.collect()[:10])

[(0, ('M', 0, 51)), (1, ('M', 0, 17)), (2, ('M', 0, 83)), (3, ('M', 0, 97)), (4, ('M', 0, 20)), (5, ('M', 0, 52)), (6, ('M', 0, 85)), (7, ('M', 0, 65)), (8, ('M', 0, 42)), (9, ('M', 0, 8))]
[(0, ('N', 0, 73)), (0, ('N', 1, 58)), (0, ('N', 2, 23)), (0, ('N', 3, 5)), (0, ('N', 4, 65)), (0, ('N', 5, 95)), (0, ('N', 6, 70)), (0, ('N', 7, 55)), (0, ('N', 8, 59)), (0, ('N', 9, 28))]


In [10]:
# print(N.collect()[:10])

[(0, ('N', 0, 73)), (0, ('N', 1, 58)), (0, ('N', 2, 23)), (0, ('N', 3, 5)), (0, ('N', 4, 65)), (0, ('N', 5, 95)), (0, ('N', 6, 70)), (0, ('N', 7, 55)), (0, ('N', 8, 59)), (0, ('N', 9, 28))]


接著使用join的方法，形成(2)式中的結構。

In [19]:
P = M.join(N)

執行第二次map，形成(3)式中的結構，這個map function由於較為簡單，不必像第一個map一樣還要另外定義，使用python中的隱函數定義法就足夠了。這裡要感謝班上熱心同學的指導。

In [20]:
P = P.map(lambda x: ((int(x[1][0][1]),int(x[1][1][1])),int(x[1][0][2])*int(x[1][1][2]) ) ) 

使用reducer1形成(4)的結構。

In [21]:
P = P.reduceByKey(reducer1)

# 三. 儲存檔案與後續處理

這樣一來就大功告成了。不過，由於spark是分散演算後再併再一起，可以將它按照行數與列數排序。接著，為了存成txt檔，我們需要將(4)的結構轉換為
$$\bigg[(i,k,P_{ik}),......\bigg]-(5)$$
為了取消tuple，可以用隱函數的方法，將(4)中每一個值都直接挑出來。

In [22]:
#P = P.sortByKey() $ sort the result
P = P.map(lambda x : (x[0][0],x[0][1],x[1])) # cancel the tuple

用collect()可以看看結果，只是會跑很久。

In [11]:
#x = P.collect()

要注意到rdd檔才可以存成output，list是不行的。

In [23]:
P.saveAsTextFile("Output")

最後，在工作資料夾內會生成一個Output資料夾，內有part-0000與part-0001兩個檔案。將兩者的副檔名改為.txt後就可以用記事本打開。裡面就是如同(5)的結構，代表第$i$列$k$行中$P_{ij}$的值